In [1]:
import os, sys, glob
import logging
import numpy as np
import matplotlib
from matplotlib.colors import LogNorm
from matplotlib import pyplot as plt

from scipy.optimize import minimize

import numba

import utils
from signal_generator import SignalGenerator
from background_generator import BackgroundGenerator

In [43]:
class pointsource_analysis:
    def __init__(self, angular_window_deg=10):
        public_data_directory = "../gxn/gXn_simulations/icecube_10year_ps/"

        aeffs = sorted(glob.glob(os.path.join(public_data_directory, 
                                              "irfs/IC86_II_effectiveArea.csv")))
        smearing = sorted(glob.glob(os.path.join(public_data_directory, 
                                                 "irfs/IC86_II_smearing.csv")))
        uptime = [os.path.join(public_data_directory, "uptime/IC86_II_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_III_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_IV_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_V_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_VI_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_VII_exp.csv")]
        data = [os.path.join(public_data_directory, "events/IC86_II_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_III_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_IV_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_V_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_VI_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_VII_exp.csv")]

        self.sig_gen = SignalGenerator(aeffs, smearing, uptime,
                            angular_resolution_scale=1.0)
        self.bg_gen = BackgroundGenerator(data)
        self.angular_window_deg = angular_window_deg

        return
    
    def one_trial(self, dec_deg ,ra_deg, integrated_flux):
        signal = self.sig_gen.generate_pointsource(declination_deg = dec_deg, 
                                    right_ascension_deg = ra_deg,
                                    integral_flux_function = integrated_flux)
        background = self.bg_gen.scramble()
        print(len(background), len(signal))
        return np.concatenate((background.to_records(),
                               signal.to_records()))
        
    def _trim(self, events, dec_deg, ra_deg, angle_deg=10):
        dist = utils.ang_dist(np.radians(ra_deg),
                      np.radians(dec_deg),
                      events['ra'], events['dec'])
        return events[dist < np.radians(angle_deg)]
        
    def calculate_sob(self, events, dec_deg, ra_deg, integrated_flux):
        S = self.sig_gen.pdf_pointsource(events,
                                          declination_deg = dec_deg, 
                                          right_ascension_deg = ra_deg,
                                          integral_flux_function = integrated_flux)
        Snllh = np.log(S)
        Bnllh = self.bg_gen.logpdf(events['logE'], np.sin(events['dec'])) - np.log(2*np.pi) 
        sob = Snllh-Bnllh 
        sob[~np.isfinite(sob)] = sob[np.isfinite(sob)].max()
        return sob
    
    def fit(self, events, dec_deg, ra_deg, integrated_flux):
        events = self._trim(events, dec_deg, ra_deg, self.angular_window_deg)
        print(len(events))
        def ts(x):
            ns, gamma = x
            f = lambda emin, emax: integrated_flux(emin, emax, 
                                                   normalization = 1,
                                                   index = gamma)
            sob = np.exp(self.calculate_sob(events, dec_deg, ra_deg, f))
            llh = np.log(ns/len(events) * (sob-1) + 1)
            #llh = np.nan_to_num(llh)
            return -llh.sum()
        
        results = minimize(ts, [1, -2], method='SLSQP',
                           bounds = [(0, len(events)), (-4, -1)])
        return results.x, results.fun
    

In [44]:
x = pointsource_analysis(angular_window_deg=10)

In [45]:
def powerlaw_integral_flux(emin, emax, normalization=1e-15, E0=1000, index=-2):
    if index > 0:
        logging.error("Please give a spectral index below 0")
        sys.exit(1)

    integral = normalization/E0**index
    if index == -1:  integral *= np.log(emax/emin)
    elif index != -1:
        integral *= (emax**(1+index) - emin**(1+index))/(1+index)
    return integral
dec_deg, ra_deg = 0, 10

gen_args = [dec_deg, ra_deg, powerlaw_integral_flux]
fit_args = [dec_deg, ra_deg, powerlaw_integral_flux]

# Produce a trial!
events = x.one_trial(*gen_args)

# Calculate the sob!
#sob = x.calculate_sob(events, *fit_args)

# Fit it!
x.fit(events, *fit_args)

#fig, ax = plt.subplots()
#ax.scatter(events['ra'], events['dec'], 
#           s = 16, 
#           marker='o',
#           alpha=0.25
#           )

#ax.set_xlim(np.radians(ra_deg-5), np.radians(ra_deg+5))
#ax.set_ylim(np.radians(dec_deg-5), np.radians(dec_deg+5))
#print(events)
#print(events.dtype.names)

761162 14
8190


/data/i3home/mjlarson/.local/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log
/cvmfs/icecube.opensciencegrid.org/py3-v4.1.0/Ubuntu_18.04_x86_64/lib/python3.7/site-packages/scipy/optimize/slsqp.py:63: RuntimeWarning: invalid value encountered in subtract
  jac[i] = (func(*((x0+dx,)+args)) - f0)/epsilon


ValueError: zero-size array to reduction operation maximum which has no identity